# TSP Art
[TSP art](https://www2.oberlin.edu/math/faculty/bosch/tspart-page.html) is an algorithmic way to create line drawings from images that can be drawn without lifting the pencil.

It is produced by creating a point cloud with a density depending on the brightness of the image and connecting the dots by a shortest [TSP](https://en.wikipedia.org/wiki/Travelling_salesman_problem) tour.

The version in this example is leaving out some of the more compelling parts, in particular the use of proper [stippling](https://en.wikipedia.org/wiki/Stippling). Follow the link above to learn more about TSP art.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx as nx

from itertools import product, combinations
from math import sqrt

from gurobipy import GRB

# GraphILP API: import networkx graphs and use travelling salesman problem
import sys
sys.path.append("../..")

from graphilp.imports import networkx as nximp
from graphilp.network import tsp
from graphilp.network.heuristics import tsp_christofides

In [ ]:
from matplotlib import image as mpimg 
from matplotlib import pyplot as plt

import numpy as np

## Create graph from image

In [ ]:
image = mpimg.imread("images/example_tsp_art.png")

In [ ]:
plt.imshow(image);

### Convert image to gray-scale

In [ ]:
# TODO: This is copied from somewhere around the web; do your own
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
gray = rgb2gray(image)

In [ ]:
plt.imshow(gray, cmap='gray');

### Draw sample from intensity distribution

In [ ]:
flat = gray.flatten()

In [ ]:
sample_index = np.random.choice(a=flat.size, replace=False, p=flat/flat.sum(), size=1000)

In [ ]:
adjusted_index = np.unravel_index(sample_index, gray.shape)

In [ ]:
plt.imshow(gray, cmap='gray')
plt.scatter(adjusted_index[1], adjusted_index[0], s=2);

## Create graph from samples

In [ ]:
def distance(indexes, i, j):
    return sqrt((indexes[0][i] - indexes[0][j])**2 + (indexes[1][i] - indexes[1][j])**2)

In [ ]:
edges = [(i, j,
         {'weight':distance(adjusted_index, i, j)}) for i, j in combinations(range(len(adjusted_index[0])), 2)]

In [ ]:
G = nx.Graph()

In [ ]:
G.add_edges_from(edges)

In [ ]:
optG = nximp.read(G)

## Create warmstart using Christofides's algorithm

In [ ]:
warmstart_tour, lower_bound = tsp_christofides.getHeuristic(optG)

In [ ]:
plt.scatter(adjusted_index[0], adjusted_index[1], s=2)
for edge in warmstart_tour:
    plt.plot((adjusted_index[0][edge[0]], adjusted_index[0][edge[1]]),
             (adjusted_index[1][edge[0]], adjusted_index[1][edge[1]]), 'r');

## Setup optimisation problem

In [ ]:
m = tsp.createModel(optG, direction=GRB.MINIMIZE, warmstart=warmstart_tour)

## Find optimal solution

In [ ]:
m.optimize()

In [ ]:
tour = gen_path_atsp.extractSolution(optG, m)

## Visualise solution

In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(121)
plt.axis('off')
plt.imshow(image)
ax = plt.subplot(122)
plt.axis('off')
ax.set_aspect(image.shape[0]/image.shape[1])
for edge in tour:
    plt.plot((adjusted_index[1][edge[0]], adjusted_index[1][edge[1]]),
             (image.shape[0]-adjusted_index[0][edge[0]], image.shape[0]-adjusted_index[0][edge[1]]), 'k');